In [1]:
import os
import sys
sys.path.insert(1,'/home/bwhiteak/ChaosESN/ESN_utils/')

import numpy as np
import matplotlib.pyplot as plt
import rc_tools as rct
import rc_matrices as rcm
import rc_analysis as rca
from scipy.optimize import fsolve
import time

import pdb

In [2]:
def rank_curve(cplus, tols):
    rank_tols = []
    for i in tols:
        rank_tols.append(rca.rank(cplus, i))
    return np.array(rank_tols)

In [7]:
sizes = [10,20,30,40,50,100,200,300,400,500,600,700,800,900,1000]
target_path = ['../Targets/Xsignal.npy',
              '../Targets/Zsignal.npy',
              '../Targets/MGsignal.npy',
              '../Targets/L3Dsignal.npy']
model_dir = ['../X_Saved_Models',
             '../Z_Saved_Models',
             '../MG_Saved_Models',
             '../L3D_Saved_Models']
sig_str = ['X','Z','MG','L3D']
SIGNUM = 3
K=1
tols = [1/10**x for x in range(-30,30)]
reps = 1
rank_arr = np.zeros((reps, len(sizes)))

for rp in range(reps):
    counter = 0
    for n in sizes:
        forcing = np.load(target_path[SIGNUM], allow_pickle=True)
        wnet_dir = 'best/Wnet'
        wnet_str  = sig_str[SIGNUM]+f'_{n}_Wnet.npy'
        fwpath = os.path.join(model_dir[SIGNUM],wnet_dir,wnet_str)
        
        param_dir = 'best/Params'
        param_str = sig_str[SIGNUM]+f'_{n}_Params.npy'
        fppath = os.path.join(model_dir[SIGNUM],param_dir,param_str)
        try:
            p = np.load(fppath, allow_pickle=True)
        except:
            counter += 1
            continue
        
        Wr, Wi = rca.get_mats(fwpath, K,n)
        a,g = p[0],p[2]
        x0 = np.zeros((n,1))
        u0 = np.zeros((K,1))
        A = rca.leaky_jacobian(x0, u0, a, g, Wi, Wr)
        B = rca.partial_u(x0, u0, a, g, Wi, Wr)
        Cn = np.nan_to_num(rca.reachable_matrix(A,B))
        if(SIGNUM != 3): # Square Cn
            Cn = Cn/np.max(np.abs(rca.eig_spectrum(Cn)))
        else:            # Non-square Cn
            Cn = Cn/np.max(np.abs(np.linalg.svd(Cn, compute_uv=False)))

        rkc = rank_curve(Cn, tols)
        v = np.argmax(np.gradient(rkc))-1
        ave_rank = (rkc[v]+rkc[v+1])//2
        print(f'Ave rank for N={n} is {ave_rank}')
        rank_arr[rp, counter] = ave_rank
        counter += 1
print('Saving to... ',
      f'./NBoutputs/Rmm{sig_str[SIGNUM]}')
np.save(f'./NBoutputs/Rmm{sig_str[SIGNUM]}', rank_arr)

Ave rank for N=10 is 1
Ave rank for N=20 is 2
Ave rank for N=30 is 2
Ave rank for N=40 is 2
Ave rank for N=50 is 7
Ave rank for N=100 is 4
Ave rank for N=200 is 38
Ave rank for N=300 is 119
Ave rank for N=400 is 3
Ave rank for N=500 is 254
Ave rank for N=600 is 149
Ave rank for N=700 is 319
Ave rank for N=800 is 92
Ave rank for N=900 is 88
Ave rank for N=1000 is 120
Saving to...  ./NBoutputs/RmmL3D


### Single model rank

In [ ]:
tols = [1/10**x for x in range(0,20)]
SIGNUM = 2 # MG
itr = 41
sz = 1000
ref = 0
run = 0

p = np.load(f'../MG_Saved_Models/MG_it{itr}_{sz}_{ref}_{run}_Preds.npy', allow_pickle=True)
Wr, Wi = rca.get_mats(f'../MG_Saved_Models/MG_it{itr}_{sz}_{ref}_{run}_Wnet.npy',K,sz)
#Wr = Wr/np.max(np.abs(np.linalg.eigvals(Wr)))
a,g = p[0],p[2]
x0 = np.zeros((sz,1))
u0 = np.zeros((K,1))
A = rca.leaky_jacobian(x0, u0, a, g, Wi, Wr)
B = rca.partial_u(x0, u0, a, g, Wi, Wr)
Cn = rca.reachable_matrix(A,B)
nan_flag = np.isnan(Cn).any()
inf_flag = np.isinf(Cn).any()
print(f'NaN present {nan_flag}\t inf present {inf_flag}\n')
Cn = np.nan_to_num(rca.reachable_matrix(A,B), nan=1)
if(SIGNUM != 3): # Square Cn
    Cn = Cn/np.abs(np.max(np.linalg.eigvals(Cn)))
else:            # Non-square Cn
    Cn = Cn/np.abs(np.max(np.linalg.svd(Cn, compute_uv=False)))

rkc = rank_curve(Cn, tols)
v = np.argmax(np.gradient(rkc))-1
ave_rank = (rkc[v]+rkc[v+1])//2
print(f'Ave rank for N={sz} is {ave_rank}')

In [ ]:
plt.plot(rkc)
plt.show()